<details><summary>Packages to be downloaded</summary>
Things it needs to install:  <br>

 *    Verilator- the simulator  <br>
 *    VCD - Generates waveforms from `.vcd` files  <br>
 *    Widgets - Each widget must first be generated then will be loaded when played
</details>

In [ ]:
#@markdown ## **Setup**
!cd /content
!git clone https://github.com/yne/vcd.git
!make -C /content/vcd
%cd /content/vcd
!make install
%cd /
import ipywidgets as widgets
from ipywidgets import GridspecLayout
from ipywidgets import AppLayout, Button, Layout, jslink, IntText, IntSlider
import requests

!mkdir -p /content/tmp_code
#Creates a text document using the Raw github url. This text document will be used as a .py file for imports
def import_text(text):
  url = "https://raw.githubusercontent.com/byuccl/digital_design_colab/master/Labs/dataflow_lab/files/%s" % text
  resp = requests.get(url)
  with open(text, 'wb') as f:
    f.write(resp.content)

import_text("import_all.py")
from import_all import *
import_source()
import_packages()
from simulation import *
from vcd2wd import *
!apt-get install verilator >/dev/null
!pip install --upgrade git+https://github.com/anon36424/nb_js_diagrammers.git
%load_ext nb_js_diagrammers

## **The Lab**

<h3>

<summary><b>Overview of the Project</b></summary>
 
</h3>

Because of how testbenches and Verilator works, it's important that you define all of your signals with the provided names and you define your modules with the provided names. If you used a different name you can change the name in Verilator but be careful.
 
You will implement 4 different functions. Each function will only be active when its button is pressed. If no button is pressed, the LEDs should show which switches are flipped.

Each function will take the input (switches) and change it. This will be displayed on the LEDs.


#### ***Function 1:***


Down Button (btnd): Shift all the bits of the input to the left 3 times when you press the down button.  
If the button is not pressed, the LEDs should show the value of the switches.  
Input your code below for this function. When ready run it and it will be linted then tested. If it doesn't work, make changes and try again.

In [ ]:
#@title Launch Simulation Workspace
createSimulationWorkSpace("tmp_code/function1")

In [ ]:
#@title Create WaveDrom
df2wd("function1")

In [ ]:
#@title Show WaveDrom
%%wavedrom_magic -h 200 -o /content/tmp_code/function1.txt
---

In [ ]:
#@title Verilator TestBench
!verilator --cc function1.sv 
!verilator -Wall --trace -cc function1.sv --exe tb_function1.cpp
!make -C obj_dir -f Vfunction1.mk Vfunction1 > /dev/null
!./obj_dir/Vfunction1

#### ***Function 2:***


Left Button (btnl): Using all 16 switches and LEDs, do a bitwise XOR with the left and right half of the input. Set the left half of the output to 0, and the right half as the result of the XOR  

If the button is not pressed, the LEDs should show the value of the switches.


In [ ]:
#@title Launch Simulation Workspace
createSimulationWorkSpace("tmp_code/function2")

In [ ]:
#@title Create WaveDrom
df2wd("function2")

In [ ]:
#@title Show WaveDrom
%%wavedrom_magic -h 200 -o /content/tmp_code/function2.txt
---

In [ ]:
#@title Verilator TestBench
!verilator -Wall -cc function2.sv --exe tb_function2.cpp
!make -C obj_dir -f Vfunction2.mk Vfunction2 > /dev/null
!./obj_dir/Vfunction2

#### ***Function 3:***


Down button (btnr): Implement the following function while btnr is high:  
![picture](https://raw.githubusercontent.com/anon36424/digital_design_colab/main/Dataflow/media/function3.png)
  
If btnr is low, then have the LED be equal to sw.

NOTE: A will be the first switch, B the second and C the third. F will be the first LED

Hint: `A = sw[0]`. You may also create intermediate signals if you prefer
  

If the button is not pressed, the LEDs should show the value of the switches.


In [ ]:
#@title Launch Simulation Workspace

createSimulationWorkSpace("tmp_code/function3")

In [ ]:
#@title Create WaveDrom
df2wd("function3")


In [ ]:
#@title Show WaveDrom
%%wavedrom_magic -h 200 -o /content/tmp_code/function3.txt
---

In [ ]:
#@title Verilator TestBench
!verilator -Wall -cc function3.sv --exe tb_function3.cpp
!make -C obj_dir -f Vfunction3.mk Vfunction3 > /dev/null
!./obj_dir/Vfunction3

#### ***Function 4:***


Up Button btnu:  
The door to the digital design lab has an electronic lock. When someone swipes their ID card, it could be a student or a professor. If it is a student, and it is between 7am and 10pm, the door will unlock. If it is a professor, the door will open no matter what time it is. Implement this functionality when the up button is pressed.

**Inputs:**  
Sw[0] - is student?  
Sw[1] - is professor?  
Sw[2] - 0 = am, 1 = pm  
Sw[6:3] - binary representation of the hour of the day  
**Output:**  
If the door unlocks, turn on all LEDs

**Note:** If the student input and the professor input are both high, the door should stay locked (this should not be possible—it is an imposter). If you receive an invalid time (ex: 15:00am) the door should also stay locked.  


If the button is not pressed, the LEDs should show the value of the switches.


In [ ]:
#@title Launch Simulation Workspace
createSimulationWorkSpace("tmp_code/function4")

In [ ]:
#@title Create WaveDrom
df2wd("function4")


In [ ]:
#@title Show WaveDrom
%%wavedrom_magic -h 200 -o /content/tmp_code/function4.txt
---

In [ ]:
#@title Verilator TestBench
!verilator -Wall -cc function4.sv --exe tb_function4.cpp
!make -C obj_dir -f Vfunction4.mk Vfunction4 > /dev/null
!./obj_dir/Vfunction4

#### ***Bringing it together***





Now add all of your functions together. When a certain button is pressed it should implement that function. When no button is pressed, it the leds should show the value of the switches.

You will just combine all of your functions together.

The module should be called `dataflow_sv`
It should have `sw` as a 16 bit wide inputs.
It should have `btnl`, `btnd`, `btnr`, and `btnu` as single bit inputs.
It should have `led` as a 16 bit wide output

You will have to rename and refactor some of your inputs. For example, on Function 3, instead of having A, B, and C be inputs, you will need to assign them as intermediate signals:
```
assign A = sw[2];
assign B = sw[1];
assign C = sw[0];
```

This is because one input is mapped to one signal.

You should use if/else if/else branches to switch between your 4 functions.


| Module Name: |dataflow_sv|||
| ----------- | ----------- |--|--|
| Port Name      | Direction       |Width|Function|
|btnd 	|Input 	|1| Activates Function1	|
|btnl 	|Input 	|1| Activates Function2	|
|btnr 	|Input 	|1| Activates Function3	|
|btnu 	|Input 	|1| Activates Function4	|
|sw 	|Input 	|16 |	The value of the switches|
|led| 	Output 	|16| The value to display on the LEDs|

In [ ]:
#@title Hint
%%html

<style>
div.warn {    
    color: #356A89;
    background-color: #D4EAF7;
    border-left: 5px solid #3C82E3;
    padding: 0.5em;
    }
 </style>
<div class=warn>
The module should be called dataflow. It should have sw as a 16 bit wide inputs. <br> 
It should have btnl, btnd, btnr, and btnu as single bit inputs. It should have led as a 16 bit wide output.
</div>


In [ ]:
#@title Launch Simulation Workspace
createSimulationWorkSpace("tmp_code/dataflow")

#### ***XDC Files:***




XDC stands for Xilinx Design Constraints. The master XDC file  has each of the ports for input and output on the Basys3 board. The XDC file connects signals with physical hardware. This will be needed to bind signals to switches, LEDs and buttons. 

Uncomment the lines containing `btnr`, `btnl`, `btnu`, and `btnd`. This tells the FPGA what input is linked to button.

***NOTE***: In future labs, if you're clever about how you name your signals in your top level module, you won't need to rename any signals in the .xdc but will just uncomment the lines that you need.

You will need to uncomment the lines that contain the button signals.
The lines look like this:  
`# set_property -dict { PACKAGE_PIN U18   IOSTANDARD LVCMOS33 } [get_ports { btnc }];`

Then run the codeblock to save your constraints file.

In [ ]:
#@title Generate an XDC File
%%bash -c 'cat > /content/tmp_code/xdc.xdc'

## Buttons
# set_property -dict { PACKAGE_PIN U18   IOSTANDARD LVCMOS33 } [get_ports { btnc }];
set_property -dict { PACKAGE_PIN T18   IOSTANDARD LVCMOS33 } [get_ports { btnu }];
set_property -dict { PACKAGE_PIN W19   IOSTANDARD LVCMOS33 } [get_ports { btnl }];
set_property -dict { PACKAGE_PIN T17   IOSTANDARD LVCMOS33 } [get_ports { btnr }];
set_property -dict { PACKAGE_PIN U17   IOSTANDARD LVCMOS33 } [get_ports { btnd }];

## Switches
set_property -dict { PACKAGE_PIN V17   IOSTANDARD LVCMOS33 } [get_ports { sw[0] }];
set_property -dict { PACKAGE_PIN V16   IOSTANDARD LVCMOS33 } [get_ports { sw[1] }];
set_property -dict { PACKAGE_PIN W16   IOSTANDARD LVCMOS33 } [get_ports { sw[2] }];
set_property -dict { PACKAGE_PIN W17   IOSTANDARD LVCMOS33 } [get_ports { sw[3] }];
set_property -dict { PACKAGE_PIN W15   IOSTANDARD LVCMOS33 } [get_ports { sw[4] }];
set_property -dict { PACKAGE_PIN V15   IOSTANDARD LVCMOS33 } [get_ports { sw[5] }];
set_property -dict { PACKAGE_PIN W14   IOSTANDARD LVCMOS33 } [get_ports { sw[6] }];
set_property -dict { PACKAGE_PIN W13   IOSTANDARD LVCMOS33 } [get_ports { sw[7] }];
set_property -dict { PACKAGE_PIN V2    IOSTANDARD LVCMOS33 } [get_ports { sw[8] }];
set_property -dict { PACKAGE_PIN T3    IOSTANDARD LVCMOS33 } [get_ports { sw[9] }];
set_property -dict { PACKAGE_PIN T2    IOSTANDARD LVCMOS33 } [get_ports { sw[10] }];
set_property -dict { PACKAGE_PIN R3    IOSTANDARD LVCMOS33 } [get_ports { sw[11] }];
set_property -dict { PACKAGE_PIN W2    IOSTANDARD LVCMOS33 } [get_ports { sw[12] }];
set_property -dict { PACKAGE_PIN U1    IOSTANDARD LVCMOS33 } [get_ports { sw[13] }];
set_property -dict { PACKAGE_PIN T1    IOSTANDARD LVCMOS33 } [get_ports { sw[14] }];
set_property -dict { PACKAGE_PIN R2    IOSTANDARD LVCMOS33 } [get_ports { sw[15] }];

## LEDs
set_property -dict { PACKAGE_PIN U16   IOSTANDARD LVCMOS33 } [get_ports { led[0] }];
set_property -dict { PACKAGE_PIN E19   IOSTANDARD LVCMOS33 } [get_ports { led[1] }];
set_property -dict { PACKAGE_PIN U19   IOSTANDARD LVCMOS33 } [get_ports { led[2] }];
set_property -dict { PACKAGE_PIN V19   IOSTANDARD LVCMOS33 } [get_ports { led[3] }];
set_property -dict { PACKAGE_PIN W18   IOSTANDARD LVCMOS33 } [get_ports { led[4] }];
set_property -dict { PACKAGE_PIN U15   IOSTANDARD LVCMOS33 } [get_ports { led[5] }];
set_property -dict { PACKAGE_PIN U14   IOSTANDARD LVCMOS33 } [get_ports { led[6] }];
set_property -dict { PACKAGE_PIN V14   IOSTANDARD LVCMOS33 } [get_ports { led[7] }];
set_property -dict { PACKAGE_PIN V13   IOSTANDARD LVCMOS33 } [get_ports { led[8] }];
set_property -dict { PACKAGE_PIN V3    IOSTANDARD LVCMOS33 } [get_ports { led[9] }];
set_property -dict { PACKAGE_PIN W3    IOSTANDARD LVCMOS33 } [get_ports { led[10] }];
set_property -dict { PACKAGE_PIN U3    IOSTANDARD LVCMOS33 } [get_ports { led[11] }];
set_property -dict { PACKAGE_PIN P3    IOSTANDARD LVCMOS33 } [get_ports { led[12] }];
set_property -dict { PACKAGE_PIN N3    IOSTANDARD LVCMOS33 } [get_ports { led[13] }];
set_property -dict { PACKAGE_PIN P1    IOSTANDARD LVCMOS33 } [get_ports { led[14] }];
set_property -dict { PACKAGE_PIN L1    IOSTANDARD LVCMOS33 } [get_ports { led[15] }];



## **Compiling with the F4PGA Toolchain**


### Installing the Toolchain

Click play. This may take up to 4 minutes.

In [ ]:
!apt install -y git wget xz-utils

In [ ]:
%%bash
cd /content
git clone https://github.com/chipsalliance/f4pga-examples
cd /content/f4pga-examples
time wget https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh -O conda_installer.sh


In [ ]:
#Creates the Conda Environment
%%bash
cd /content/f4pga-examples
export F4PGA_INSTALL_DIR=~/opt/f4pga
export FPGA_FAM=xc7
time bash conda_installer.sh -u -b -p $F4PGA_INSTALL_DIR/$FPGA_FAM/conda;
time source "$F4PGA_INSTALL_DIR/$FPGA_FAM/conda/etc/profile.d/conda.sh";
time conda env create -f $FPGA_FAM/environment.yml

In [ ]:
#Download F4PGA Arch Defs
%%bash
export F4PGA_INSTALL_DIR=~/opt/f4pga
export FPGA_FAM=xc7
source "$F4PGA_INSTALL_DIR/$FPGA_FAM/conda/etc/profile.d/conda.sh";
conda activate xc7
mkdir -p $F4PGA_INSTALL_DIR/xc7/install

F4PGA_TIMESTAMP='20220920-124259'
F4PGA_HASH='007d1c1'
export F4PGA_PACKAGES='install-xc7 xc7a50t_test'

for PKG in $F4PGA_PACKAGES; do
  wget -qO- https://storage.googleapis.com/symbiflow-arch-defs/artifacts/prod/foss-fpga-tools/symbiflow-arch-defs/continuous/install/${F4PGA_TIMESTAMP}/symbiflow-arch-defs-${PKG}-${F4PGA_HASH}.tar.xz | tar -xJC $F4PGA_INSTALL_DIR/${FPGA_FAM}
done

### Compiling the Lab with the Toolchain
This should take up to 3 minutes

In [ ]:
#This creates the Makefile
with open("/content/tmp_code/Makefile", "w") as f:
  f.write("""current_dir := ${CURDIR}
TARGET := basys3

TOP := dataflow_sv

XDC := ${current_dir}/*.xdc

SOURCES := ${current_dir}/dataflow.sv

include /content/f4pga-examples/common/common.mk
""")

: 

In [ ]:
#Make the project
%%bash
export F4PGA_INSTALL_DIR=~/opt/f4pga
export FPGA_FAM=xc7
export TARGET=basys3
export FOLDER=Dataflow_SV
export FILES="Makefile xdc.xdc dataflow.sv"
source "$F4PGA_INSTALL_DIR/$FPGA_FAM/conda/etc/profile.d/conda.sh";
mkdir -p /content/"$FOLDER"
cd /content/tmp_code
for file in $FILES
do 
    cp "$file" /content/"$FOLDER"/
done
conda activate xc7
cd /content/$FOLDER
cp /content/tmp_code/errorFeedback.py ./
time SURELOG_CMD="-parse" make 2> ./error.txt 1>./compile.txt
python errorFeedback.py

## **Testing it on the board**

To Download it to the board you will need OpenOCD.  

First create a folder to house the files on your local machine. Then download the OpenOCD configure file and the bitstream from this Notebook and add then to your folder (they will be in a zip file in /content/dataflow_sv.zip). On command line go into the folder and run the command   

`openocd --file Dataflow_SV.cfg`

It should only take a few moments to download to the board.

In [ ]:
#@title Create File for openocd
folder = "Dataflow_SV"

with open(f"/content/{folder}.cfg", "w") as f:
  f.write("""interface ftdi
ftdi_device_desc "Digilent USB Device"
ftdi_vid_pid 0x0403 0x6010
# channel 1 does not have any functionality
ftdi_channel 0
# just TCK TDI TDO TMS, no reset
ftdi_layout_init 0x0088 0x008b
reset_config none
adapter_khz 10000

source [find cpld/xilinx-xc7.cfg]
source [find cpld/jtagspi.cfg]
init

puts [irscan xc7.tap 0x09]

set xc7a35t "0362D093"
set xc7a100t "13631093"
set code [drscan xc7.tap 32 0]  
puts $code

if { $code == $xc7a35t} {
    puts "The board has an xc7a35t"
}

if { $code == $xc7a100t} {
    puts "The board has an xc7a100t"
}
"""
+
f"""
puts "Programming..."
pld load 0 {folder}.bit
exit"""
)

In [ ]:
#@title Create Zip
%%bash
cd /content
export FOLDER=Dataflow_SV
cp $FOLDER/build/basys3/*.bit ./
zip $FOLDER.zip *.bit $FOLDER.cfg


Click [Here](https://colab.research.google.com/github/byuccl/digital_design_colab2/blob/master/Exercises/twos_complement/twos_complement.ipynb) to move on to the Two's Complement Lesson.